## **INSTALL **

In [0]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

## **LIBRARY**

In [0]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import requests
import numpy as np

## **GET HTML SOURCE**

In [0]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(10)
driver.get("https://vivatechnology.com/startups-big-list/")

time.sleep(15)
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, 'html.parser')
driver.close()
time.sleep(0.3)

## **MAKE A STARTUPS LIST**

In [0]:
PARAM_INFO = ['company_name','location','resume','tags']

temp ={}
for param in PARAM_INFO:
  temp[param] = []

all_info = soup.find_all("div", {"class":"card-over"})
for info in all_info:
  event_location = info.find("div", {"class":"event-location"})
  company_name = info.find("strong")
  resume_temp = info.find("div", {"class":"resume"})
  tags_temp = info.find("div", {"class":"tags"})
  
  if(event_location is None):
    temp['location'] = np.hstack((temp['location'],None))
  else:
    temp['location'] = np.hstack((temp['location'],event_location.find("span").string))
    
  if(company_name is None):
    temp['company_name'] = np.hstack((temp['company_name'],None))
    temp['resume'] = np.hstack((temp['resume'],None))
    temp['tags'] = np.hstack((temp['tags'],None))
  else:
    resume = resume_temp.find("p")
    tags = tags_temp.find_all("span")
    tags_text = ''
    for tag in tags:
      tags_text = tags_text + tag.string + ", \n"
    temp['company_name'] = np.hstack((temp['company_name'],company_name.string))
    temp['resume'] = np.hstack((temp['resume'],resume.string))
    temp['tags'] = np.hstack((temp['tags'],tags_text))

df = pd.DataFrame()
for param in PARAM_INFO:
  df[param] = temp[param]    



## **SAVE A LIST**

In [0]:
from google.colab import files
df.to_csv("compay_list_startups.csv")
files.download("compay_list_startups.csv")